Task 2 of The Forage Quantitative Research Module | JPMC

In [1]:
#imports
import datetime as dt
import pandas as pd
import numpy as np

print("imports complete")

imports complete


In [ ]:
def price_gas_storage_contract(
    injection_dates,
    withdrawal_dates,
    prices,
    daily_injection_rate,
    max_storage_volume,
    monthly_storage_cost,
    injection_withdrawal_cost_per_mmbtu
):
    """
    Prices a gas storage contract.

    Parameters
    ----------
    injection_dates : list of datetime
    withdrawal_dates : list of datetime
    prices : dict {datetime: price_per_mmbtu}
    daily_injection_rate : float (MMBtu per day)
    max_storage_volume : float (MMBtu)
    monthly_storage_cost : float ($ per month)
    injection_withdrawal_cost_per_mmbtu : float ($ per MMBtu)

    Returns
    -------
    contract_value : float
    """

    inventory = 0.0
    cashflow = 0.0

    # Sort events chronologically
    events = [(d, "inject") for d in injection_dates] + \
             [(d, "withdraw") for d in withdrawal_dates]
    events.sort(key=lambda x: x[0])

    # Track storage duration
    start_date = events[0][0]
    end_date = events[-1][0]
    months_in_storage = max(1, (end_date.year - start_date.year) * 12 +
                               (end_date.month - start_date.month))

    # Handle injections and withdrawals
    for date, event_type in events:
        price = prices[date]
        volume = daily_injection_rate

        if event_type == "inject":
            if inventory + volume > max_storage_volume:
                volume = max_storage_volume - inventory

            inventory += volume
            cashflow -= volume * price
            cashflow -= volume * injection_withdrawal_cost_per_mmbtu

        elif event_type == "withdraw":
            if inventory - volume < 0:
                volume = inventory

            inventory -= volume
            cashflow += volume * price
            cashflow -= volume * injection_withdrawal_cost_per_mmbtu

    # Storage costs
    cashflow -= monthly_storage_cost * months_in_storage

    return cashflow


In [3]:
from datetime import datetime

# Sample inputs
injection_dates = [
    datetime(2025, 6, 1),
    datetime(2025, 6, 2),
    datetime(2025, 6, 3)
]

withdrawal_dates = [
    datetime(2025, 12, 1),
    datetime(2025, 12, 2),
    datetime(2025, 12, 3)
]

prices = {
    datetime(2025, 6, 1): 2.00,
    datetime(2025, 6, 2): 2.05,
    datetime(2025, 6, 3): 2.10,
    datetime(2025, 12, 1): 3.10,
    datetime(2025, 12, 2): 3.15,
    datetime(2025, 12, 3): 3.20
}

value = price_gas_storage_contract(
    injection_dates=injection_dates,
    withdrawal_dates=withdrawal_dates,
    prices=prices,
    daily_injection_rate=1_000_000,
    max_storage_volume=3_000_000,
    monthly_storage_cost=100_000,
    injection_withdrawal_cost_per_mmbtu=0.01
)

print(f"Contract value: ${value:,.0f}")


Contract value: $2,640,000
